![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=SocialStudies/HansardAnalysis/hansard-analysis.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Sunset and Sunrise Analysis

Welcome to the wonderful world of Sunrises and Sunsets! In this vibrant notebook, we embark on breathtaking journey through the captivating beauty of nature's daily masterpieces: sunrises and sunsets.

Sunrise and sunset data was collected through the [Sunset and Sunrise](https://sunrise-sunset.org/api) api, which provides sunset and sunrise times when given a specific latitude and longitude. It is a simple REST api which requires a GET request from [here](https://api.sunrise-sunset.org/json). Other sources for sunrise and sunset times, such as the sunset and sunrise times of Calgary, can be found [here](https://www.timeanddate.com/sun/canada/calgary)

In [1]:
import requests
from datetime import *
from zoneinfo import ZoneInfo
from dateutil import tz
from pytz import timezone
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tzfpy import get_tz

In [2]:
lat= 51.049999
lng= -114.06666

query = {'lat': lat, 'lng': lng, 'date': '2023-05-18'}
response = requests.get(url = 'https://api.sunrise-sunset.org/json?', params=query)

data = response.json()
print(data)
df = pd.json_normalize(data['results'])
display(df)

{'results': {'sunrise': '11:39:40 AM', 'sunset': '3:25:45 AM', 'solar_noon': '7:32:42 PM', 'day_length': '15:46:05', 'civil_twilight_begin': '11:00:07 AM', 'civil_twilight_end': '4:05:18 AM', 'nautical_twilight_begin': '10:03:07 AM', 'nautical_twilight_end': '5:02:18 AM', 'astronomical_twilight_begin': '8:35:13 AM', 'astronomical_twilight_end': '6:30:11 AM'}, 'status': 'OK'}


,sunrise,sunset,solar_noon,day_length,civil_twilight_begin,civil_twilight_end,nautical_twilight_begin,nautical_twilight_end,astronomical_twilight_begin,astronomical_twilight_end
0,11:39:40 AM,3:25:45 AM,7:32:42 PM,15:46:05,11:00:07 AM,4:05:18 AM,10:03:07 AM,5:02:18 AM,8:35:13 AM,6:30:11 AM


Looking at the data in the _json_ format and the _dataframe_ format, we have multiple things to take note of. In particular, we see that we have access to the time of `sunrise` and `sunset` given a particular latitude and longitude. We also have access to `day_length` which highlights the length of a particular date. However, one particular issue with this dataset is that times are represented in the **UTC timezone**. Let's keep mind of this and _transform_ our data later.

In [3]:
sunrise = data['results']['sunrise']
sunset = data['results']['sunset']
print(f"Given latitude {lat} and longitude {lng}, the corresponding sunrise and sunset times are {sunrise} and {sunset} in the UTC timezone.")

Given latitude 51.049999 and longitude -114.06666, the corresponding sunrise and sunset times are 11:39:40 AM and 3:25:45 AM in the UTC timezone.


`Longitude` controls the directions of _east_ and _west_, which generally alters timezones, leading to leaps of times. `Latitude` controls the directions of _north_ and _south_, which affects the length of day and night. The rule is that the further away someone is from the equator, the greater the differences are between nighttime and daytime.

Remember our issue earlier regarding timezones? Let's solve that by transforming our time from the __UTC timezone__ to the __MST timezone__. This is a fairly simple change, as the __UTC timezone__ is 6 hours ahead of the __MST timezome__. By going back 6 hours, we are able to obtain times in our timezone. 

Later on, when you input your own `longitude` and `latitude` values, keep in mind your particular timezone in order to differentiate from the __UTC timezone__. We'll also find a solution to finding your particular timezone based off inputted `longitude` and `latitude` values. 

In [4]:
timezone_to_switch = get_tz(lng, lat)
print(timezone_to_switch)

America/Edmonton


Great, we now have a reliable way to get a timezone from inputted longitude and latitude values.

In [5]:
def time_convert(time, timezone_swap:int):
    # Parse the time string into a datetime object
    # Note, here you have to change the timezone information and adjust with a + or - accordingly
    t = datetime.strptime(time, '%I:%M:%S %p') - timedelta(hours=timezone_swap)
    # Format the datetime object into a 24-hour time string
    format = '%H:%M:%S'
    return t.strftime(format)

print(time_convert(sunrise, 6))

05:39:40


In [6]:
total_info = {"date": []}

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2023, 5, 0)
end_date = date(2023, 5, 19)

for single_date in daterange(start_date, end_date):
    total_info["date"].append(single_date.strftime("%Y-%m-%d"))
data = pd.DataFrame(total_info)

display(data)

,date
0,2023-04-17
1,2023-04-18
2,2023-04-19
3,2023-04-20
4,2023-04-21
5,2023-04-22
6,2023-04-23
7,2023-04-24
8,2023-04-25
9,2023-04-26


In [7]:
# Alter these longitude and latitude values below

longitude = -114.06666
latitude = 51.049999

timezone_to_switch = get_tz(longitude, latitude)
print(timezone_to_switch)

America/Edmonton


In [8]:
# params format: latitude, longitude, date (YYYY-MM-DD)
sunset_info = {"sunset": []}
sunrise_info = {"sunrise": []}
day_length = {"day_length": []}

def sunrisesunset(lat, lng, date):
    params = {"lat":lat, "lng":lng, "date":date}
    data = requests.get("https://api.sunrise-sunset.org/json", params=params)
    data = json.loads(data.text)
    data = data["results"]
    return data["sunrise"], data["sunset"], data["day_length"]


for i in range(len(data)):
    temp, temp1, temp2 = sunrisesunset(latitude, longitude, total_info['date'][i])
    sunrise_info['sunrise'].append(time_convert(temp, 6))
    sunset_info['sunset'].append(time_convert(temp1, 6))
    day_length['day_length'].append(temp2)

sunset_info = pd.DataFrame(sunset_info)
sunrise_info = pd.DataFrame(sunrise_info)
day_length = pd.DataFrame(day_length)

In [10]:
temp_total_info = data.join(sunrise_info['sunrise'])
temp_total_info2 = temp_total_info.join(sunset_info["sunset"])
temp_total_info3 = temp_total_info2.join(day_length['day_length'])
display(temp_total_info3)

,date,sunrise,sunset,day_length
0,2023-04-17,06:34:57,20:36:41,14:01:44
1,2023-04-18,06:32:51,20:38:20,14:05:29
2,2023-04-19,06:30:46,20:39:58,14:09:12
3,2023-04-20,06:28:43,20:41:37,14:12:54
4,2023-04-21,06:26:40,20:43:15,14:16:35
5,2023-04-22,06:24:38,20:44:53,14:20:15
6,2023-04-23,06:22:37,20:46:32,14:23:55
7,2023-04-24,06:20:37,20:48:10,14:27:33
8,2023-04-25,06:18:38,20:49:48,14:31:10
9,2023-04-26,06:16:40,20:51:26,14:34:46
